### Question (a): Loading Data

In [2]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, naive_bayes , metrics

df = pd.read_json('./all/train.json')

C:\Users\AngZhou\Anaconda3\envs\py27\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Question (b): 
Tell us about the data. How many samples (dishes) are there in the training set? How many categories (types of cuisine)? Use a list to keep all the unique ingredients appearing in the trainingset. How many unique ingredients are there?

In [3]:
num_obs = df.shape[0]
num_obs

39774

##### Answer 1: Total number of the samples is: 39774

In [4]:
cuisine_array=df.cuisine.unique()
print cuisine_array
print cuisine_array.size

[u'greek' u'southern_us' u'filipino' u'indian' u'jamaican' u'spanish'
 u'italian' u'mexican' u'chinese' u'british' u'thai' u'vietnamese'
 u'cajun_creole' u'brazilian' u'french' u'japanese' u'irish' u'korean'
 u'moroccan' u'russian']
20


##### Answer 2: There are totally 20 types of cuisines.

In [5]:
ingredients_collection=df['ingredients'].values
# print ingredients_collection

ingredients = set()
for i in ingredients_collection:
    ingredients = ingredients.union(i)
    
ing_list = list(ingredients)

In [6]:
len(ing_list)

6714

##### Answer 3: There are totally 6714 unique ingredients.

### Question (c)
Represent each dish by a binary ingredient feature vector. Suppose there are d different ingredients in total from the training set, represent each dish by a 1×d binary ingredient vector x, where xi =1 if the dish contains ingredient i and xi =0 otherwise. For example, suppose all the ingredients we have in the training set are { beef, chicken, egg, lettuce, tomato, rice } and the dish is made by ingredients { chicken, lettuce, tomato, rice }, then the dish could be represented by a 6×1 binary vector [0,1,0,1,1,1] as its feature or attribute. Use n×d feature matrix to represent all the dishes in training set and testset, where n is the number of dishes.

In [7]:
%%time

df2 = pd.read_json('./all/test.json')
num_obs2 = df2.shape[0]

print 'start'
featureList = []

for i in range(num_obs): #len(trainData)
    feature = [0] * len(ing_list)
    sub_list = ingredients_collection[i]
    for j in range(len(ing_list)):
        if ing_list[j] in sub_list:
            #print ("yes",ing_list[j],j)
            feature[j] = 1
    featureList.append(feature)

featureMatrix = np.matrix(featureList)
print featureMatrix.shape


featureList2 = [] ## for test set


for i in range(df2.shape[0]): # len(trainData)
    feature = [0] * len(ing_list)  # Note here, we still use the ingredient list obtained from training set
    sub_list = (df2['ingredients'].values)[i]
    for j in range(len(ing_list)):
        if ing_list[j] in sub_list:
            feature[j] = 1
    featureList2.append(feature)

featureMatrix2 = np.matrix(featureList2)
print featureMatrix2.shape
print 'end'

start
(39774L, 6714L)
(9944L, 6714L)
end
Wall time: 3min 1s


In [10]:
print np.sum(featureMatrix[0]) == len(df.ingredients[0])
print np.sum(featureMatrix[1]) == len(df.ingredients[1])
print np.sum(featureMatrix[2]) == len(df.ingredients[2])
print np.sum(featureMatrix[3]) == len(df.ingredients[3])
print np.sum(featureMatrix2[0]) == len(df2.ingredients[0])
print np.sum(featureMatrix2[1]) == len(df2.ingredients[1])
print np.sum(featureMatrix2[2]) == len(df2.ingredients[2])
print np.sum(featureMatrix2[3]) == len(df2.ingredients[3])

True
True
True
True
True
True
True
True


##### Answer: Above print statements check whether the numbers of the ingredients in featureMatrix match the dataset. And we check the first 4 observations with correct output.

### Question (d)
Using Naïve Bayes Classiﬁer to perform 3 fold cross-validation on the training set and report your average classiﬁcation accuracy. Try both Gaussian distribution prior assumption and Bernoulli distribution prior assumption. 

In [8]:
fold1=list(cross_validation.KFold(num_obs, n_folds=3))[0]
fold2=list(cross_validation.KFold(num_obs, n_folds=3))[1]
fold3=list(cross_validation.KFold(num_obs, n_folds=3))[2]

In [9]:
def cross_validation_Naive_Bayes_evaluate(dataSet,labels,fold_indices,NB):
    train_indices, test_indices = fold_indices
    X_train = dataSet[train_indices]
    Y_train = labels[train_indices]
    X_test = dataSet[test_indices]
    Y_test = labels[test_indices]
    
    if(NB == 'Bernoulli'):
        clf = naive_bayes.BernoulliNB()
    elif(NB == 'Gaussian'):
        clf = naive_bayes.GaussianNB()
    clf.fit(X_train, Y_train)
    # BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
    Y_estimate = clf.predict(X_test)
    return (Y_estimate,Y_test)

In [10]:
def accuracy(Y_predict,Y_true):
    return metrics.accuracy_score(Y_true, Y_predict)

In [11]:
print 'Bernoulli'
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold1,'Bernoulli')
acc1 = accuracy(Y_predict,Y_true)
print acc1
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold2,'Bernoulli')
acc2 = accuracy(Y_predict,Y_true)
print acc2
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold3,'Bernoulli')
acc3 = accuracy(Y_predict,Y_true)
print acc3

Bernoulli
0.684190677326897
0.6795142555438226
0.6869060190073918


In [12]:
(acc1+acc2+acc3)/3.0

0.6835369839593705

##### Answer: The above is the three accuracy gained by three fold cross validation based on Bernoulli Naive Bayes Classifier. The average accuracy is 0.6835369839593705.

In [13]:
print 'Gaussian'
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold1,'Gaussian')
acc1 = accuracy(Y_predict,Y_true)
print acc1
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold2,'Gaussian')
acc2 = accuracy(Y_predict,Y_true)
print acc2
(Y_predict, Y_true) = cross_validation_Naive_Bayes_evaluate(featureMatrix,df.cuisine,fold3,'Gaussian')
acc3 = accuracy(Y_predict,Y_true)
print acc3

Gaussian
0.37901644290239855
0.3829386031075577
0.37758334590435966


In [14]:
(acc1+acc2+acc3)/3

0.3798461306381053

##### Answer: The above is the three accuracy gained by three fold cross validation based on Gaussian Naive Bayes Classifier. The average accuracy is 0.3798461306381053.

### Question (e)
For Gaussian prior and Bernoulli prior, which performs better in terms of cross-validation accuracy? Why? Please give speciﬁc arguments. 

##### Answer: Bernoulli prior is much better than the Gaussian prior in terms of cross-validation accuracy. The reason is that the vector of each feature is a binary vector. Gaussian prior works better when the values of feature are continuous values.

### Question (f)
Using Logistic Regression Model to perform 3 fold cross-validation on the training set and report your average classiﬁcation accuracy. 

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
def cross_validation_Logistic_Regression_evaluate(dataSet,labels,fold_indices):
    train_indices, test_indices = fold_indices
    X_train = dataSet[train_indices]
    Y_train = labels[train_indices]
    X_test = dataSet[test_indices]
    Y_test = labels[test_indices]
        
    clf = LogisticRegression(multi_class='ovr')
    clf.fit(X_train, Y_train)
    Y_estimate = clf.predict(X_test)
    return (Y_estimate,Y_test)

In [17]:
print 'Logistic Regression'
(Y_predict, Y_true) = cross_validation_Logistic_Regression_evaluate(featureMatrix,df.cuisine,fold1)
acc1 = accuracy(Y_predict,Y_true)
print acc1
(Y_predict, Y_true) = cross_validation_Logistic_Regression_evaluate(featureMatrix,df.cuisine,fold2)
acc2 = accuracy(Y_predict,Y_true)
print acc2
(Y_predict, Y_true) = cross_validation_Logistic_Regression_evaluate(featureMatrix,df.cuisine,fold3)
acc3 = accuracy(Y_predict,Y_true)
print acc3

Logistic Regression
0.7758334590435964
0.7721375773118118
0.7786996530396741


In [18]:
(acc1+acc2+acc3)/3

0.7755568964650275

###### Answer: The above is the three accuracy gained by three fold cross validation based on Logistic Regression Classifier. The average accuracy is 0.7755568964650275.

### Question (g)
Train your best-performed classiﬁer with all of the training data, and generate test labels on testset. Submit your results to Kaggle and report the accuracy.

Start:
Based on previous 3-fold cross validation. The best-performed classiﬁer is logistic regression classifier. We should use logistic regression classifier.

#### Step 1: reading test data

In [19]:
df2 = pd.read_json('./all/test.json')

In [20]:
df2.shape

(9944, 2)

#### Step 2: extract the ingredients features based on the ingredients list we got from training data. Following the same process we did for training set.

In [21]:
print 'start'
featureList2 = []

for i in range(df2.shape[0]): #len(trainData)
    feature = [0] * len(ing_list)
    sub_list = (df2['ingredients'].values)[i]
    for j in range(len(ing_list)):
        if ing_list[j] in sub_list:
            feature[j] = 1
    featureList2.append(feature)

featureMatrix2 = np.matrix(featureList2)
print featureMatrix2.shape
print 'end'

start
(9944L, 6714L)
end


#### Step 3: build logistic regression model on training set, then apply estimation on testing set.

In [22]:
clf = LogisticRegression(multi_class='ovr')
clf.fit(featureMatrix, df.cuisine)
Y_estimate = clf.predict(featureMatrix2)

#### Step 4: output the estimation and construct the submission file.

In [23]:
# Output the solution
import csv

file_name = "submission.csv"
with open(file_name, mode='wb') as submission_file:
    fieldnames = ['id', 'cuisine']
    writer = csv.DictWriter(submission_file, fieldnames=fieldnames)
    
    writer.writerow({'id': 'id', 'cuisine': 'cuisine'})
    for i in range(len(Y_estimate)):
        writer.writerow({'id': df2['id'][i], 'cuisine': Y_estimate[i]})
    
print "done"

done


##### The Whole Problem Finished